In [1]:
import requests
from datetime import datetime, timedelta

In [3]:
def verify_transactions(address, period_hours=24):
    """
    Queries and verifies Bitcoin transactions for a given address within a specified time period.

    Parameters:
    - address (str): The Bitcoin address to check for transactions.
    - period_hours (int): The time period in hours to look back for transactions. Defaults to 24 hours.

    Returns:
    - float: The total amount of Bitcoin received by the address in the specified period.

    Raises:
    - requests.exceptions.RequestException: If there is an issue with the network request.
    """

    # API Endpoint for getting raw address transactions
    api_url = f"https://blockchain.info/rawaddr/{address}"
    
    amount = 0
    
    try:
        # Request transaction data for the address
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an error if the request failed

        # Parse the JSON response
        data = response.json()
        transactions = data.get('txs', [])

        # Get the current time and the oldest allowed timestamp
        current_time = datetime.utcnow()
        period_delta = timedelta(hours=period_hours)
        oldest_timestamp = current_time - period_delta
        
        # Check transactions within the specified period
        for tx in transactions:
            tx_time = datetime.utcfromtimestamp(tx['time'])  # Convert Unix timestamp to datetime

            if tx_time > oldest_timestamp:
                for output in tx['out']:
                    if output.get('addr') == address:
                        amount_btc = output['value'] / 1e8  # Convert Satoshis to BTC
                        amount += amount_btc
                        sending_addresses = [input['prev_out']['addr'] for input in tx['inputs'] if 'prev_out' in input]
                        print(f"Transaction Time: {tx_time}")
                        print(f"Amount Received: {amount_btc} BTC")
                        print(f"Sent from addresses: {', '.join(sending_addresses)}\n")
                        
        if not transactions:
            print(f"No transactions to address {address} in the last {period_hours} hours.")

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
    
    return amount

def get_bitcoin_price():
    """
    Fetches the current price of Bitcoin in USD.

    Returns:
    - float: The current price of Bitcoin in USD, or None if there was an error fetching the price.

    Raises:
    - requests.exceptions.RequestException: If there is an issue with the network request.
    """

    # CoinGecko API URL for Bitcoin price
    api_url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
    
    try:
        # Make a request to the CoinGecko API
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an error if the request failed

        # Parse the JSON response
        price_info = response.json()
        
        # Extract the Bitcoin price in USD
        bitcoin_price = price_info['bitcoin']['usd']
        
        print(f"The current price of Bitcoin is: ${bitcoin_price} USD")
        
        return bitcoin_price

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching the Bitcoin price: {e}")
        
        return None

In [22]:
# Example usage:
# Replace 'to_address' with the actual Bitcoin address you're interested in
to_address = '3FupZp77ySr7jwoLYEJ9mwzJpvoNBXsBnE'

# Example usage:
amount_BTC = verify_transactions(to_address)
amount_BTC

Transaction Time: 2023-11-06 13:00:59
Amount Received: 26.996 BTC
Sent from addresses: bc1quq29mutxkgxmjfdr7ayj3zd9ad0ld5mrhh89l2, bc1quq29mutxkgxmjfdr7ayj3zd9ad0ld5mrhh89l2, bc1quq29mutxkgxmjfdr7ayj3zd9ad0ld5mrhh89l2, bc1quq29mutxkgxmjfdr7ayj3zd9ad0ld5mrhh89l2, bc1qyszzdl6wfy44w279jgwp0jetld2udpdqwl4232



26.996

In [23]:
# Example usage
BTC_price = get_bitcoin_price()
BTC_price

The current price of Bitcoin is: $35029 USD


35029

In [24]:
BTC_price * amount_BTC

945642.884